
![ImageName](pic/title.jpg)  

**作者：[lqy](https://www.heywhale.com/home/user/profile/5f27fd9633e1be002cc65a1d)** 华东师范大学气象学研究生、和鲸社区气象数据科学频道版主

🐋：本项目来自和鲸社区《[气象训练营⑦：WRF模式后处理](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1)》活动，所有教案代码都可以一键跑通，你可以 fork 后在线在线运行、调试学习、完成作业练习。  

学习过程中如果你遇到任何问题，欢迎使用搜索引擎，或在 [讨论区](https://www.heywhale.com/home/competition/forumlist/64478fec113e81a18dc70cd1) 中发帖提出，我们很乐意为你提供帮助。

# 关卡六：绘制WRF模拟的天气图

天气分析中，天气图是非常重要的。天气图包含了许多物理要素，**如风向、气压、温度、湿度、降水等，这些要素是天气预报的基础**。天气图的分析可以帮助预报员更好地**理解天气系统的演变和发展趋势，从而提高预报的准确性**。

## 一、什么是天气图

天气图是填有**各地同一时间气象观测记录的特种地图**，它描述了**某一瞬间某一区域的天气状况**。运用天气学原理，对天气图连续分析和研究，就可获得天气过程发展的规律，从而做出天气预报。因此，天气图是制作天气预报的基本工具，**可以分为地面图和高空图**。

> 中央气象台天气图：http://www.nmc.cn/publish/observations/china/dm/weatherchart-h000.htm

  
![Image Name](pic/weathermap.png)  


由于**天气形成、发展、消亡都具有一定的连贯性**，我们可以通过分析连续几天或者几时次的天气图平判断某**天气系统的移动速度、强度变化，以及可能有新的系统生成**等等。  





## 二、绘制地面天气图

地面天气图是一种用于描述地面天气状况的图表，通常包括气压、温度、湿度、风向和风速等信息。

### 导入模块

In [ ]:
# 一次性导入所有模块
from wrf import uvmet, to_np, getvar, interplevel, smooth2d, get_cartopy, cartopy_xlim, cartopy_ylim, latlon_coords
import numpy as np
import pandas as pd
from datetime import datetime
from netCDF4 import Dataset
import xarray as xr
from metpy.units import units
from metpy.calc import height_to_geopotential, geopotential_to_height
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import NaturalEarthFeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import shapely.geometry as sgeom
import cmaps
from glob import glob
from copy import copy

import warnings
warnings.filterwarnings('ignore')

### 标注高压和低压中心

我们需要找到从气压数据中找到高压系统和低压系统，并在其中心进行标注。

In [ ]:
# 参考来源：https://unidata.github.io/python-gallery/examples/HILO_Symbol_Plot.html

# 标注高压系统中心为H，低压系统中心为L
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k', plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=25,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                '\n' + str(np.int(data[mxy[i], mxx[i]])),
                color=color, size=25, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

### 添加经纬度标签

由于Cartopy尚不能对非等经纬投影添加经纬度标签，因此我们需要自定义函数来额外实现这一功能。

In [ ]:
'''
Andrew Dawson 提供了解决方法: https://gist.github.com/ajdawson/dd536f786741e987ae4e
'''
# 由于Cartopy尚不能对非等经纬投影添加经纬度标签，因此我们需要自定义函数来额外实现这一功能。
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])

# 在兰伯特投影的底部X轴上绘制刻度线
def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])

# 在兰伯特投影的左侧y轴上绘制刻度线
def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])

# 在兰伯特投影的左侧y轴上绘制刻度线
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible: 
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

### WRF数据读取与变量提取

In [ ]:
# WRF数据目录
wrfout_path = '/home/mw/input/typhoon9537/'

# 用 netCDF4 包读取 WRF 模拟数据
wrf_file = Dataset(wrfout_path + '/wrfout_d01_2019-08-08_18_00_00')

# 提取海平面气压、10m高度的u和v风速
slp = getvar(wrf_file, 'slp', timeidx=0)
u10 = getvar(wrf_file, 'U10', timeidx=0)
v10 = getvar(wrf_file, 'V10', timeidx=0)

# 提取WRF模拟的经纬度数组
lats, lons = latlon_coords(slp)

# 提取WRF模拟的地图投影
wrf_proj = get_cartopy(slp)

### 地面天气图绘制

In [ ]:
# 创建画布
fig = plt.figure(figsize=(10,8))
# 设置地图投影
ax = plt.axes(projection=wrf_proj)
# 设置地图范围
ax.set_xlim(cartopy_xlim(slp))
ax.set_ylim(cartopy_ylim(slp))

# 读取国界线和九段线
province = shpreader.Reader('/home/mw/input/data5246/中国地图/China_provinces/China_provinces.shp')
nineline = shpreader.Reader('/home/mw/input/data5246/中国地图/China_10-dash_line/China_10-dash_line.shp')
# 绘制国界线和九段线
ax.add_geometries(province.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5, 
                  edgecolor='k',
                  facecolor='none',
                  zorder=2)
ax.add_geometries(nineline.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5,
                  color='k',
                  zorder=2)

# 绘制海平面气压（等值线）
plt.contour(to_np(lons), 
            to_np(lats), 
            to_np(slp), 
            levels=np.arange(930,1020,5),
            extend='both',
            colors='black', 
            transform=ccrs.PlateCarree())

# 绘制海平面气压（用contourf方法实现等值线填充）
plt.contourf(to_np(lons), 
            to_np(lats), 
            to_np(slp), 
            levels=np.arange(930,1020,5),
            extend='both',
            transform=ccrs.PlateCarree(),
            cmap=cmaps.MPL_BuPu_r)

# 标注低压系统中心L
plot_maxmin_points(to_np(lons)[10:-10,10:-10],
                   to_np(lats)[10:-10,10:-10],
                   to_np(slp)[10:-10,10:-10],
                   'min', 200, symbol='L', color='white', 
                   transform=ccrs.PlateCarree())

# 绘制10m风场
# dict(half=2, full=4, flag=20)表示短线代表风速2m/s，长线代表风速4m/s，旗帜代表风速20m/s
ax.barbs(to_np(lons)[::25,::25], to_np(lats)[::25,::25], 
         to_np(u10)[::25,::25], to_np(v10)[::25,::25], 
         length=8, 
         linewidth=0.5, 
         barb_increments=dict(half=2, full=4, flag=20),
         transform=ccrs.PlateCarree())

# 添加colorbar
cbar = plt.colorbar(ax=ax, extend='both', shrink=1)
cbar.set_label('Pressure (hPa)', fontdict={'size':20})
cbar.ax.tick_params(labelsize=20)

# 添加经纬度网格线
ax.gridlines(color='black', linestyle='dotted')
# 添加经纬度标签
xticks = list(np.arange(100,140,2))
yticks = list(np.arange(20,40,2))
fig.canvas.draw()
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)

# 设置刻度标签的字体大小
plt.tick_params(labelsize=15)
# 添加标题
plt.title(str(slp.Time.values)[0:16]+'(UTC)', loc='left', fontsize=20)
plt.show()

### ✍小练习：调节平滑效果

平滑可以有效地**去除天气数据中的噪声和异常值**，使得数据更加**平滑和稳定**，从而更**容易进行分析和预测**。  

> **smooth2d函数：https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.smooth2d.html**  
  
平滑本质上是一种滤波，smooth2d函数使用的平滑核是  
  
$$\begin{split}\frac{1}{4 + cenweight} * \begin{bmatrix}  
                       0 & 1 & 0 \\  
                       1 & cenweight & 1 \\  
                       0 & 1 & 0  
                     \end{bmatrix}\end{split}$$  


In [ ]:
### （你的代码）###




### ✍小练习：绘制一张高空图

**高空天气图形式更为多样，反映的信息也更为丰富，选择一种你感兴趣的高空图进行绘制**  

![Image Name](pic/gaokong.png)

In [ ]:
### （你的代码）###




恭喜你完成了WRF后处理训练营第六关的学习材料，能够顺利地**绘制出 WRF 模拟的地面和高空天气图**，可以感受一下预报员是如何分析天气过程的~~~

## 闯关题  

### STEP1：根据要求完成题目：  


1. 根据案例中绘制 WRF 模拟的地面天气图，台风中心等压线具有什么特点？  
A. 密集  
B. 稀疏  
 

2. 绘制 WRF 模拟的高空天气图，台风中心的气温与周围环境温度相比？  
A. 更高  
B. 无明显变化  
C. 更低  


3. 基于ERA5再分析资料（数据位于`/home/mw/input/ERA5_Lekima4742/ERA5_Lekima.nc`），绘制 500 hPa 高空天气图，副热带高压位于台风的哪里？  
A. 东南  
B. 东北  
C. 西南  
D. 西北  



In [ ]:
# 填入你的答案
answer_1 = ''
answer_2 = ''
answer_3 = '' 

## ⚡ 下一关预告：  

下一关我们将介绍如何绘制 WRF 模拟的剖面图，深入分析台风利奇马的空间结构模拟~~